In [1]:
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [2]:
MARKDOWN_SOURCE = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [6]:
class MarkdownToGoogleDoc:
    def __init__(self):
        self.service = None
        self.doc_id = None
        self.requests = []
        self.current_index = 1  # Google Docs index starts at 1
        self.is_footer_section = False

    def authenticate(self):
        """Handles Google Colab Authentication"""
        try:
            # Checking if already authenticated to avoid redundant prompts
            try:
                from google.auth import default
                default()
            except Exception:
                print("Authenticating user...")
                auth.authenticate_user()

            self.service = build('docs', 'v1')
            print("Authentication successful.")
        except Exception as e:
            print(f"Authentication failed: {e}")
            raise

    def create_document(self, title):
        """Creates a new blank Google Doc"""
        try:
            doc = self.service.documents().create(body={'title': title}).execute()
            self.doc_id = doc.get('documentId')
            print(f"Created document with ID: {self.doc_id}")
            return self.doc_id
        except HttpError as e:
            print(f"Error creating document: {e}")
            raise

    def _calculate_indent(self, line):
        """Calculates nesting level based on leading spaces (2 spaces = 1 level)"""
        stripped = line.lstrip()
        spaces = len(line) - len(stripped)
        return spaces // 2

    def parse_markdown(self, text):
        """Parses markdown text and builds API requests"""
        # Reset requests and index for a clean run if called multiple times
        self.requests = []
        self.current_index = 1
        self.is_footer_section = False

        lines = text.strip().split('\n')

        for line in lines:

            # Handle Separator (Footer detection)
            if line.strip() == '---':
                self.is_footer_section = True
                continue

            # 1. Headers
            if line.startswith('#'):
                level = len(line.split(' ')[0])
                # Ensure we only strip the leading hash and space, not internal text
                content = line.lstrip('#').strip() + '\n'
                style = f'HEADING_{level}' if level <= 6 else 'NORMAL_TEXT'

                # Insert Text
                self._insert_text(content)
                # Apply Style
                self._apply_paragraph_style(style, self.current_index, self.current_index + len(content))

                self.current_index += len(content)

            # 2. Checkboxes (FIXED ORDER HERE)
            elif '- [ ]' in line:
                content = line.replace('- [ ]', '').strip() + '\n'

                # STEP 1: Insert the text FIRST so indices exist
                self._insert_text(content)

                # STEP 2: Apply styles to the now-existing text
                self._handle_mentions(content, self.current_index)

                # STEP 3: Apply checkbox bullet style
                self._create_bullet(
                    'BULLET_CHECKBOX',
                    self.current_index,
                    self.current_index + len(content),
                    nesting_level=0
                )
                self.current_index += len(content)

            # 3. Bullet Points
            elif line.strip().startswith(('*', '-')):
                indent_level = self._calculate_indent(line)
                content = re.sub(r'^[\s\*\-]*', '', line).strip() + '\n'

                self._insert_text(content)
                self._create_bullet(
                    'BULLET_DISC_CIRCLE_SQUARE',
                    self.current_index,
                    self.current_index + len(content),
                    nesting_level=indent_level
                )
                self.current_index += len(content)

            # 4. Standard Text / Footer
            else:
                if not line.strip():
                    content = '\n' # Preserve empty lines
                else:
                    content = line + '\n'

                self._insert_text(content)

                if self.is_footer_section and line.strip():
                    self._apply_footer_style(self.current_index, self.current_index + len(content))

                self.current_index += len(content)

    def _insert_text(self, text):
        """Helper to create InsertTextRequest"""
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text
            }
        })

    def _apply_paragraph_style(self, style_type, start, end):
        """Helper to style headers"""
        self.requests.append({
            'updateParagraphStyle': {
                'range': {'startIndex': start, 'endIndex': end},
                'paragraphStyle': {'namedStyleType': style_type},
                'fields': 'namedStyleType'
            }
        })

    def _create_bullet(self, preset, start, end, nesting_level=0):
        """Helper to create bullets or checkboxes"""
        self.requests.append({
            'createParagraphBullets': {
                'range': {'startIndex': start, 'endIndex': end},
                'bulletPreset': preset,
            }
        })

    def _handle_mentions(self, text, start_index):
        """Finds @mentions and styles them"""
        mention_pattern = r'(@\w+)'
        for match in re.finditer(mention_pattern, text):
            # Calculate absolute indices
            m_start = start_index + match.start()
            m_end = start_index + match.end()

            self.requests.append({
                'updateTextStyle': {
                    'range': {'startIndex': m_start, 'endIndex': m_end},
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {
                            'color': {'rgbColor': {'red': 0.1, 'green': 0.1, 'blue': 0.8}}
                        }
                    },
                    'fields': 'bold,foregroundColor'
                }
            })

    def _apply_footer_style(self, start, end):
        """Styles the footer (grey, italic)"""
        self.requests.append({
            'updateTextStyle': {
                'range': {'startIndex': start, 'endIndex': end},
                'textStyle': {
                    'italic': True,
                    'fontSize': {'magnitude': 9, 'unit': 'PT'},
                    'foregroundColor': {
                        'color': {'rgbColor': {'red': 0.4, 'green': 0.4, 'blue': 0.4}}
                    }
                },
                'fields': 'italic,fontSize,foregroundColor'
            }
        })

    def execute_batch(self):
        """Sends all requests to Google Docs API"""
        if not self.requests:
            return

        try:
            self.service.documents().batchUpdate(
                documentId=self.doc_id,
                body={'requests': self.requests}
            ).execute()
            print("Formatting applied successfully.")
        except HttpError as e:
            print(f"Error applying formatting: {e}")

In [7]:
def main():
    converter = MarkdownToGoogleDoc()

    # 1. Auth
    converter.authenticate()

    # 2. Create Doc
    title = "Product Team Sync - Generated"
    doc_id = converter.create_document(title)

    # 3. Parse & Plan
    converter.parse_markdown(MARKDOWN_SOURCE)

    # 4. Execute
    converter.execute_batch()

    print(f"\nSUCCESS! Access your document here: https://docs.google.com/document/d/{doc_id}")

if __name__ == "__main__":
    main()

Authentication successful.


Created document with ID: 1WDUoqPRZPpSFLIr-3azcwlRZS7A4SEBo2s84d0AFkbU
Formatting applied successfully.

SUCCESS! Access your document here: https://docs.google.com/document/d/1WDUoqPRZPpSFLIr-3azcwlRZS7A4SEBo2s84d0AFkbU
